In [1]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

In [1]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import torch.backends.cudnn as cudnn
import random

Using cache found in /home/M2021080/.cache/torch/hub/pytorch_vision_main
/home/M2021080/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /home/M2021080/.cache/torch/hub/pytorch_vision_main
/home/M2021080/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# train_loader, test_loader = CIFAR100.get_data(58*3)
train_loader, test_loader = CIFAR100.get_data(128, use_original=True)

Files already downloaded and verified
Files already downloaded and verified


/home/M2021080/anaconda3/envs/cuda_11_3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
import models
teacher = torch.load("saved_models/resnet20/best.pth")
student = models.__dict__['resnet20'](num_classes=100)

In [4]:
# from Models import Conv

# depth = 18

# model = torch.load(f"saved_models/resnet/resnet{depth}.pth").module
# teacher = Conv.resnet_feature(100, depth, model)
# student = Conv.resnet_feature(100, depth, pretrained="IMAGENET1K_V1")


In [5]:
device = "cuda"

teacher = teacher.to(device)
# teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1])

student = student.to(device)
# student = torch.nn.DataParallel(student, device_ids=[0, 1])


In [6]:
criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))
criterion_response2 = lambda a,b : criterion_CE(torch.softmax(a/4, dim=1),torch.softmax(b/4, dim=1))


In [7]:
S_optimizer = optim.SGD(student.parameters(), lr=0.05, momentum=0.9)
T_optimizer = optim.SGD(teacher.parameters(), lr=0.05, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()

In [8]:
S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)

In [9]:
utils.set_seed()

best_acc = 0.0
stack = 0

accs_train = []
accs_test = []

In [10]:
utils.test(teacher, test_loader,device)
utils.test(student, test_loader,device)


100%|██████████| 79/79 [00:08<00:00,  9.78it/s]


0 	 test acc : 0.6866999864578247


100%|██████████| 79/79 [00:05<00:00, 13.29it/s]

0 	 test acc : 0.011399999260902405


tensor(0.0114, device='cuda:0')

In [11]:
import tqdm.notebook as tq


In [ ]:
student_test_accs = []
layer_num = 4

utils.set_seed()
for epoch in range(100):
    
    print(f"lr : {S_scheduler.get_last_lr()}")
    if S_scheduler.get_last_lr()[0] < 0.000001:
        break
        
    T_correct = 0
    S_correct = 0
    all_data = 0
    
    loss_distill = []
    loss_CE = []
    loss_response = []
    student.train()
    teacher.eval()
    with tqdm(train_loader, unit='batch') as pbar:
        for img, label in train_loader:
            input_data = img.to(device)
            label = label.to(device)


            all_data += len(input_data)
            input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=0.5, std = 0.01, mult = 0.4).cuda()

            S_optimizer.zero_grad()
            T_optimizer.zero_grad()

            layer = random.randint(0,  layer_num)
    #         output_s, fk = student(input_data,layer)
    #         output_t, fk_lrp = teacher(input_lrp,layer)
            output_s = student(input_data)
            output_t = teacher(input_lrp)

    #         channal wise pooling
    #         fk = torch.mean(fk, dim=2)
    #         fk_lrp = torch.mean(fk_lrp, dim=2)

    #         distill_loss = mse(fk, fk_lrp)
            distill_loss = 0

            CE_loss = criterion_CE(output_s, label)

            response_loss = criterion_response2(output_t, output_s)

            T_correct += sum(label == torch.argmax(output_t, dim=1))
            S_correct += sum(label == torch.argmax(output_s, dim=1))

            loss_CE.append(CE_loss.item())
    #         loss_distill.append(distill_loss.item())
            loss_response.append(response_loss.item())

            loss = (distill_loss * 3 + CE_loss + response_loss) / 2 # 82.03
            loss.backward()
            S_optimizer.step()
            pbar.set_postfix(TeacaherACC = (T_correct / all_data).item(), StudentACC = (S_correct / all_data).item())
    #     print("distill loss : ", sum(loss_distill) / len(loss_distill))
    print("general loss : ", sum(loss_CE) / len(loss_CE))
    print("response loss : ", sum(loss_response) / len(loss_response))

    print(f"Teacher acc: {T_correct / all_data}")
    print(f"Student acc: {S_correct / all_data}")

    test_acc = utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함

    if test_acc > best_acc + 0.01:
        stack = 0
        best_acc = test_acc

    else:
        stack+=1

    if stack > 3:  
        S_scheduler.step()
        stack = 0

    student_test_accs.append(test_acc.item())
    print("=" * 100)

lr : [0.05]


  0%|          | 0/391 [01:35<?, ?batch/s, StudentACC=0.0755, TeacaherACC=0.947] 


general loss :  4.03482528781647
response loss :  4.603915214538574
Teacher acc: 0.9467399716377258
Student acc: 0.07547999918460846


100%|██████████| 79/79 [00:05<00:00, 13.29it/s]


0 	 test acc : 0.11919999867677689
lr : [0.05]


...

100%|██████████| 79/79 [00:07<00:00, 10.44it/s]


20 	 test acc : 0.531499981880188
lr : [0.05]


  0%|          | 0/391 [01:41<?, ?batch/s, StudentACC=0.579, TeacaherACC=0.947]


general loss :  1.4760570550513694
response loss :  4.574433730386407
Teacher acc: 0.9471799731254578
Student acc: 0.579039990901947


 65%|██████▍   | 51/79 [00:04<00:01, 20.60it/s]

In [ ]:
# distill loss를 2배 키워보는것도 좋을지도

In [11]:
utils.test(teacher, test_loader,device, epoch) # student도 변하는거 확인 완료함
utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함


100%|██████████| 27/27 [00:06<00:00,  4.23it/s]


32 	 test acc : 0.7930999994277954


100%|██████████| 27/27 [00:06<00:00,  4.38it/s]

32 	 test acc : 0.8247999548912048


tensor(0.8248, device='cuda:0')

In [14]:
torch.save(student, "saved_models/resnet/resnet{depth}_student.pth")

In [15]:
import json

with open(f"saved_models/resnet/resnet{depth}.json", "w") as f:
    json.dump({"student_test_accs" : student_test_accs}, f)

In [ ]:
# from Models import Conv

# depth = 101

# model = torch.load(f"saved_models/vgg/vgg{depth}.pth").module
# teacher = Conv.resnet_feature(100, depth, model)
# student = Conv.resnet_feature(100, depth, pretrained="IMAGENET1K_V1")


# device = "cuda"

# teacher = teacher.to(device)
# teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1])

# student = student.to(device)
# student = torch.nn.DataParallel(student, device_ids=[0, 1])

# criterion_onlylabel = lambda a,b : mse(a*b, b)
# criterion_CE = nn.CrossEntropyLoss()
# mse = nn.MSELoss()
# softmax = torch.nn.Softmax(dim = 1)
# criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
# criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))

# S_optimizer = optim.SGD(student.parameters(), lr=0.05, momentum=0.9)
# T_optimizer = optim.SGD(teacher.parameters(), lr=0.05, momentum=0.9)
# CE_loss = nn.CrossEntropyLoss()

# S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)
# T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)



# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# np.random.seed(0)
# cudnn.benchmark = False
# cudnn.deterministic = True
# random.seed(0)
# best_acc = 0.0
# stack = 0

# accs_train = []
# accs_test = []

# utils.test(teacher, test_loader,device)
# utils.test(student, test_loader,device)
